# Домашнее задание 6

Перед выполнением заданий необходимо сохранить копию этого блокнота себе на диск. Или скачать `.ipynb`/`.py` файл для локальной разработки.

За работу можно получить максимум 8 баллов.

Вы можете сдавать эту работу как на R, так и на Python.

# Задание 1 (3 балла)

В [папке](https://drive.google.com/drive/folders/1kwbJk5jottoSMep2rDEefeE_26-LQp1D?usp=sharing) вы можете найти данные для работы.

Необходимо объединить таблицы таким образом, чтобы вывести информацию с ответами на следующие вопросы:
* по какой услуге клиенты чаще всего оставляли запросы?
* сколько запросов поступило от женщин и мужчин?
* вывести информацию о том, по какому региону по какой услуге было больше всего запросов? вывести информацию в порядке убывания общего числа запросов, указать название самой популярной услуге (если их несколько) и в алфавитном порядке по региону

In [ ]:
# ваш код здесь
import pandas as pd
branch_df = pd.read_csv(r"C:\Users\Admin\Desktop\Задание 1\branch.csv")
client_df = pd.read_csv(r"C:\Users\Admin\Desktop\Задание 1\client.csv")
manager_df = pd.read_csv(r"C:\Users\Admin\Desktop\Задание 1\manager.csv")
questionnaire_df = pd.read_csv(r"C:\Users\Admin\Desktop\Задание 1\questionnaire.csv")
service_df = pd.read_csv(r"C:\Users\Admin\Desktop\Задание 1\service.csv")

merged_df = pd.merge(
    questionnaire_df,
    client_df,
    on ='client_id',
    how ='left'
)
merged_df = pd.merge(
    merged_df,
    service_df,
    on ='service_id',
    how ='left'
)
merged_df = pd.merge(
    merged_df,
    branch_df,
    on ='branch_id',
    how ='left'
)

service_counts = merged_df.groupby('service_name').size() \
    .reset_index(name='request_count').sort_values('request_count', ascending=False)
max_requests = service_counts['request_count'].max()
most_popular_services = service_counts[service_counts['request_count'] == max_requests]
print("Максимальное количество запросов:", max_requests)
for _, row in most_popular_services.iterrows():
    print(f" {row['service_name']}")
print()

merged_df['gender_group'] = merged_df['gender']
gender_counts = merged_df.groupby('gender_group').size().reset_index(name='request_count')
gender_counts = gender_counts.sort_values('request_count', ascending=False)
print(gender_counts)
print()

region_service_counts = merged_df.groupby(['branch_name', 'service_name']).size() \
    .reset_index(name='service_count')
total_by_region = merged_df.groupby('branch_name').size().reset_index(name='total_requests')
region_stats = pd.merge(
    region_service_counts,
    total_by_region,
    on='branch_name',
    how='left'
)
region_stats_sorted = region_stats.sort_values(
    ['branch_name', 'service_count', 'service_name'],
    ascending=[True, False, True]
)
most_popular_per_region = region_stats_sorted \
    .groupby('branch_name') \
    .first() \
    .reset_index()
max_counts = region_stats \
    .groupby('branch_name')['service_count'] \
    .max() \
    .reset_index()
max_counts.columns = ['branch_name', 'max_service_count']
region_stats_with_max = pd.merge(
    region_stats,
    max_counts,
    on='branch_name'
)
most_popular_all = region_stats_with_max \
    [region_stats_with_max['service_count'] == region_stats_with_max['max_service_count']]
most_popular_all = most_popular_all.sort_values(
    ['total_requests', 'branch_name'],
    ascending=[False, True]
)
for _, row in most_popular_all.iterrows():
    print(f"{row['branch_name']}")
    print(f" {row['total_requests']}, {row['service_name']}")

# Задание 2 (5 баллов)

В [папке](https://drive.google.com/drive/folders/16JRLbMGbNc36eHe8wG4isHUgfQzu7n0z?usp=sharing) вы можете найти данные для работы.


В датасете представлена информация о продаже компьютерных игр. Можно представить, что ваша задача - собрать самые интересные факты про рынок компьютерных игр на основе этого датасета.

Визуализировать различными способами минимум три признака, а также минимум три взаимодействия между признаками.


In [ ]:
# ваш код здесь или в других ячейках
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.core.interchange.dataframe_protocol import DataFrame

games_df = pd.read_csv(r"C:\Users\Admin\Desktop\video_games_sales.csv")

games_num = games_df['Name'].count()
years_num =  games_df['Year_of_Release'].nunique()
genres_num = games_df['Genre'].nunique()

df_clean = games_df.dropna(subset=['Year_of_Release'])
df_clean['Year_of_Release'] = df_clean['Year_of_Release'].astype(int)
games_per_year = df_clean.groupby('Year_of_Release')['Name'].count()
games_genre = games_df.groupby('Genre')['Name'].count()
games_rating = games_df.groupby('Rating')['Name'].count()

plt.bar(["количество игр, тыс", "количество лет", "количество жанров"], [games_num/1000, years_num, genres_num])
plt.show()

plt.plot(games_per_year.index, games_per_year.values)
plt.show()
plt.figure(figsize=(12, 4))
plt.bar(games_genre.index, games_genre.values)
plt.show()
plt.bar(games_rating.index, games_rating.values)
plt.show()